# finetune DistilGPT-2 and use it for inference

## Causal language modeling
Causal language models are frequently used for text generation

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
from datasets import load_dataset

eli5 = load_dataset("eli5_category", split="train[:5000]", trust_remote_code=True)


Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

In [9]:
eli5 = eli5.train_test_split(test_size=0.3)

In [11]:
eli5["test"][5]

{'q_id': '5mdezf',
 'title': 'What happens if an animal who is supposed to hibernate is unable to?',
 'selftext': "What happens if an animal who is supposed to hibernate during the winter is unable to because of health reasons or even environmental factors (i.e. home being destroyed by forest fire or being snatched out of environment by poachers)? Does its body wear down and fall into a deep sleep regardless of the severe circumstances? Does the animal eventually get over the need to hibernate after pushing through the normal sleep cycle? Does the animal risk death if it doesn't hibernate?",
 'category': 'Biology',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dc2oqcj', 'dc3erwp'],
  'text': ['A particular example: brown bears. If the bear doesn\'t get enough chow to get fat before hibernation, it\'s likely to wake up in the middle of the winter and go about looking for anything to eat. What you get is a шатун, a "rocker", which is willing to kill anything that gets in its 

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
eli5 = eli5.flatten()
eli5["test"][5]

{'q_id': '5mdezf',
 'title': 'What happens if an animal who is supposed to hibernate is unable to?',
 'selftext': "What happens if an animal who is supposed to hibernate during the winter is unable to because of health reasons or even environmental factors (i.e. home being destroyed by forest fire or being snatched out of environment by poachers)? Does its body wear down and fall into a deep sleep regardless of the severe circumstances? Does the animal eventually get over the need to hibernate after pushing through the normal sleep cycle? Does the animal risk death if it doesn't hibernate?",
 'category': 'Biology',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dc2oqcj', 'dc3erwp'],
 'answers.text': ['A particular example: brown bears. If the bear doesn\'t get enough chow to get fat before hibernation, it\'s likely to wake up in the middle of the winter and go about looking for anything to eat. What you get is a шатун, a "rocker", which is willing to kill anything that gets in i

In [ ]:
# preprocessing function to join the list of strings for each example and tokenize the result

def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [17]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/3500 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1071 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1380 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1213 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1029 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1500 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1234 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1469 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2145 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1543 > 1024). Running this sequence through the model will result in indexing errors


You can now use a second preprocessing function to

concatenate all the sequences
split the concatenated sequences into shorter chunks defined by block_size, which should be both shorter than the maximum input length and short enough for your GPU RAM.
Copied


In [18]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it 
    # instead of this drop, you can customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/3500 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1500 [00:00<?, ? examples/s]

In [20]:
# Use the end-of-sequence token as the padding token

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [21]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [22]:
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_8246/4176279854.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.904600,3.856858
2,3.822200,3.847629
3,3.778500,3.846014


TrainOutput(global_step=3402, training_loss=3.8443856842023076, metrics={'train_runtime': 552.9197, 'train_samples_per_second': 49.19, 'train_steps_per_second': 6.153, 'total_flos': 888343627825152.0, 'train_loss': 3.8443856842023076, 'epoch': 3.0})

In [23]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 46.81


In [24]:
prompt = "Somatic hypermutation allows the immune system to"

In [27]:
from transformers import pipeline

generator = pipeline("text-generation", model="Proxiii/my_awesome_eli5_clm-model")
generator(prompt)


config.json:   0%|          | 0.00/977 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': 'Somatic hypermutation allows the immune system to adapt to the stimulus of one stimulus to another. Thus, this can be done using different methods, which is why some patients prefer to use more stimulus than others. However, another reason why patients perceive'}]

In [28]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Proxiii/my_awesome_eli5_clm-model")
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [29]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Proxiii/my_awesome_eli5_clm-model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [30]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['Somatic hypermutation allows the immune system to activate this system to destroy itself. It doesn\'t stop the organism from getting used to your body, but when it finally stops turning on your body, it also causes the body to react with that same "good" immune system. It\'s also something we can look into in the lab at this point in our life. A person with a high risk of a stroke may not be able to successfully complete their stroke. We just need to look into it in other ways to develop our "normal" blood']